---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

def blight_model():
    

    # load data
    train_data = pd.read_csv('train.csv',encoding='cp1252')
    test_data = pd.read_csv('test.csv')
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    # delete useless rows from train_data
    train_data = train_data[(train_data['compliance'] == 0) | (train_data['compliance'] == 1)]
    
    # add coordinates to data
    latlons = latlons.set_index('address').join(addresses.set_index('address'))
    train_data = train_data.set_index('ticket_id').join(latlons.set_index('ticket_id'))
    test_data = test_data.set_index('ticket_id').join(latlons.set_index('ticket_id'))

    # select features to use
    train_data = train_data[['agency_name', 'state', 'disposition', 'lon', 'lat', 'fine_amount',
                             'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'compliance']]

    test_data = test_data[['agency_name', 'state', 'disposition', 'lon', 'lat', 'fine_amount',
                           'admin_fee', 'state_fee', 'late_fee', 'discount_amount']]


    # transform categorical features to one-hot
    train_data = pd.get_dummies(train_data, columns=['agency_name', 'state', 'disposition'])
    test_data = pd.get_dummies(test_data, columns=['agency_name', 'state', 'disposition'])

    # fill nan values
    train_data.lon.fillna(method='pad', inplace=True)
    train_data.lat.fillna(method='pad', inplace=True)
    test_data.lon.fillna(method='pad', inplace=True)
    test_data.lat.fillna(method='pad', inplace=True)

    # features, common for train_data and test_data
    common_features = list(train_data.columns.intersection(test_data.columns))
    
    # leave olny useful data
    y_train = train_data['compliance']
    X_train = train_data[common_features]
    X_test = test_data[common_features]

    # features scaling
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # fit model with best hyper-parameters
    clf = MLPClassifier(solver='adam', random_state=0)
    grid_values = {'hidden_layer_sizes':[[64, 16], [128, 32], [256, 64]],
                   'alpha':[0.01, 0.1, 0.5, 1, 3]}

    grid = GridSearchCV(clf, param_grid=grid_values, scoring='roc_auc').fit(X_train, y_train)
    
    # produce output
    test_data = pd.read_csv('test.csv').set_index('ticket_id')
    test_data['compliance'] = grid.predict_proba(X_test)[:,1]
    output = test_data['compliance']

    return output


blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.045374
285362    0.017810
285361    0.044094
285338    0.046655
285346    0.052846
285345    0.046727
285347    0.056183
285342    0.426426
285530    0.015759
284989    0.036193
285344    0.056240
285343    0.018524
285340    0.018575
285341    0.056104
285349    0.040226
285348    0.038087
284991    0.041452
285532    0.033124
285406    0.028350
285001    0.022503
285006    0.014755
285405    0.027355
285337    0.018025
285496    0.053046
285497    0.044093
285378    0.027264
285589    0.035403
285585    0.039155
285501    0.049925
285581    0.017738
            ...   
376367    0.016722
376366    0.026574
376362    0.231330
376363    0.243120
376365    0.016722
376364    0.026574
376228    0.029830
376265    0.025654
376286    0.299743
376320    0.028842
376314    0.030386
376327    0.327990
376385    0.323790
376435    0.324717
376370    0.324431
376434    0.053704
376459    0.054507
376478    0.007281
376473    0.029668
376484    0.027484
376482    0.024877
37